In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Model Evaluation Notebook\n",
    "\n",
    "This notebook provides comprehensive evaluation of all models in the facial recognition pipeline:\n",
    "- Face Detection Performance\n",
    "- Feature Extraction Quality\n",
    "- Recognition Accuracy\n",
    "- Additional Models (Age, Sentiment, Pose, etc.)\n",
    "\n",
    "## Setup and Dependencies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.metrics import confusion_matrix, classification_report\n",
    "\n",
    "from src.core.face_detector import FaceDetector\n",
    "from src.core.feature_extractor import FeatureExtractor\n",
    "from src.models.face_recognition.recognition_model import FaceRecognitionModel\n",
    "from src.utils.image_processing import load_image, preprocess_image"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Face Detection Evaluation\n",
    "\n",
    "Evaluate the face detector's performance on various scenarios:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def evaluate_face_detector(test_images_path):\n",
    "    detector = FaceDetector()\n",
    "    results = []\n",
    "    \n",
    "    for img_path in test_images_path:\n",
    "        img = load_image(img_path)\n",
    "        faces = detector.detect(img)\n",
    "        results.append({\n",
    "            'image': img_path,\n",
    "            'faces_detected': len(faces),\n",
    "            'confidence_scores': [face['confidence'] for face in faces]\n",
    "        })\n",
    "    \n",
    "    return pd.DataFrame(results)\n",
    "\n",
    "# Example usage:\n",
    "# detector_results = evaluate_face_detector(['path/to/test/images'])\n",
    "# detector_results.describe()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Feature Extraction Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def analyze_feature_vectors(feature_extractor, image_paths):\n",
    "    features = []\n",
    "    for img_path in image_paths:\n",
    "        img = load_image(img_path)\n",
    "        feat = feature_extractor.extract(img)\n",
    "        features.append(feat)\n",
    "    \n",
    "    features = np.array(features)\n",
    "    \n",
    "    # Analyze feature distribution\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    sns.heatmap(features[:10], cmap='viridis')\n",
    "    plt.title('Feature Vector Heatmap (First 10 Samples)')\n",
    "    plt.show()\n",
    "    \n",
    "    return features"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Recognition Model Performance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def evaluate_recognition_model(model, test_data):\n",
    "    predictions = []\n",
    "    true_labels = []\n",
    "    \n",
    "    for img, label in test_data:\n",
    "        pred = model.predict(img)\n",
    "        predictions.append(pred)\n",
    "        true_labels.append(label)\n",
    "    \n",
    "    # Generate confusion matrix\n",
    "    cm = confusion_matrix(true_labels, predictions)\n",
    "    plt.figure(figsize=(10, 8))\n",
    "    sns.heatmap(cm, annot=True, fmt='d')\n",
    "    plt.title('Confusion Matrix')\n",
    "    plt.show()\n",
    "    \n",
    "    # Print classification report\n",
    "    print(classification_report(true_labels, predictions))\n",
    "    \n",
    "    return predictions, true_labels"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Pipeline Integration Testing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def test_full_pipeline(image_path):\n",
    "    # Initialize components\n",
    "    detector = FaceDetector()\n",
    "    feature_extractor = FeatureExtractor()\n",
    "    recognition_model = FaceRecognitionModel()\n",
    "    \n",
    "    # Process image\n",
    "    img = load_image(image_path)\n",
    "    faces = detector.detect(img)\n",
    "    \n",
    "    results = []\n",
    "    for face in faces:\n",
    "        features = feature_extractor.extract(face['crop'])\n",
    "        identity = recognition_model.predict(features)\n",
    "        results.append({\n",
    "            'bbox': face['bbox'],\n",
    "            'confidence': face['confidence'],\n",
    "            'identity': identity\n",
    "        })\n",
    "    \n",
    "    return results"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4